In [1]:
import scipy.io
import numpy as np

import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import AdamW  # Use PyTorch's AdamW

from transformers import BertConfig, BertModel
from torch import nn

import matplotlib.pyplot as plt

from transformers import get_scheduler

from tqdm.notebook import tqdm

import os

from transformers import PreTrainedModel
from transformers.modeling_outputs import BaseModelOutput

from csibert_util import CSIBERT

W0920 14:03:35.231000 82559 site-packages/torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [2]:
# Check MPS availability
if torch.backends.mps.is_available():
    print("MPS backend is available!")
else:
    print("MPS backend is not available.")

MPS backend is available!


In [3]:
# Check for MPS device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

Using device: mps


In [4]:
# Load Multi-Cell CSI Data
# multi_cell_data = scipy.io.loadmat('foundation_model_data/multi_cell_csi.mat')['csiMatrix']

# Load Mobility CSI Data
# mobility_data = scipy.io.loadmat('foundation_model_data/mobility_csi.mat')['csiTimeVarying']

# Load MU-MIMO CSI Data
# mu_mimo_data = scipy.io.loadmat('foundation_model_data/mu_mimo_csi.mat')['csiMU']

# Load Heterogeneous Network Data
# heterogeneous_data = scipy.io.loadmat('foundation_model_data/heterogeneous_network_csi.mat')
# csi_macro = heterogeneous_data['csiMacro']
# csi_small_cell = heterogeneous_data['csiSmallCell']

cell_data = scipy.io.loadmat('foundation_model_data/csi_data_massive_mimo.mat')['multi_cell_csi']

In [5]:
# Function to preprocess individual CSI matrices
def preprocess_csi_matrix(csi_matrix):
    """
    Preprocess CSI data to handle variable-length sequences and retain time, frequency, and spatial dimensions.
    """
    csi_real = np.real(csi_matrix)
    csi_imag = np.imag(csi_matrix)
    
    # Normalize real and imaginary parts
    csi_real_normalized = (csi_real - np.mean(csi_real)) / np.std(csi_real)
    csi_imag_normalized = (csi_imag - np.mean(csi_imag)) / np.std(csi_imag)

    # Combine real and imaginary components
    csi_combined = np.stack([csi_real_normalized, csi_imag_normalized], axis=-1)  # Shape: (subcarriers, Tx, Rx, 2)
    
    # Flatten freq, spatial, and real/imaginary into a single feature dimension
    time_dim = csi_combined.shape[0]
    feature_dim = np.prod(csi_combined.shape[1:])  # Tx × Rx × 2
    csi_combined = csi_combined.reshape(time_dim, feature_dim)  # Shape: (time, feature_dim)
    
    return csi_combined

# Traverse the nested cell structure
preprocessed_data = []
sequence_lengths = []

for cell_idx in range(cell_data.shape[0]):  # Iterate over cells
    for ue_idx in range(cell_data.shape[1]):  # Iterate over UEs
        ue_data = cell_data[cell_idx, ue_idx]
        for scenario in ue_data[0]:  # Each UE has multiple scenarios
            processed_csi = preprocess_csi_matrix(scenario)
            preprocessed_data.append(processed_csi)
            sequence_lengths.append(processed_csi.shape[0])  # Track sequence lengths

# Convert to padded 3D array (batch_size, sequence_length, feature_dim)
max_sequence_length = max(sequence_lengths)
feature_dim = preprocessed_data[0].shape[-1]

# Pad sequences dynamically
padded_data = np.zeros((len(preprocessed_data), max_sequence_length, feature_dim), dtype=np.float32)
attention_masks = np.zeros((len(preprocessed_data), max_sequence_length), dtype=np.float32)
for i, sequence in enumerate(preprocessed_data):
    seq_len = sequence.shape[0]
    padded_data[i, :seq_len, :] = sequence
    attention_masks[i, :seq_len] = 1  # Mask for unpadded tokens

# Mask data for masked signal prediction task
def mask_data(data, mask_ratio=0.15):
    mask = np.random.rand(*data.shape[:-1]) < mask_ratio  # Exclude the last dimension (real/imaginary parts)
    masked_data = np.copy(data)
    masked_data[mask, :] = 0  # Replace masked elements with 0
    return masked_data, mask

masked_data, mask = mask_data(padded_data)

# Ensure consistent data types
masked_data = masked_data.astype(np.float32)
attention_masks = torch.tensor(attention_masks, dtype=torch.float32).to(device)

In [6]:
cell_data.shape

(10, 200)

In [7]:
batch_size = 32

train_dataset = TensorDataset(
    torch.tensor(masked_data).float(),  # Masked inputs
    torch.tensor(padded_data).float(),  # Labels (original data)
    torch.tensor(attention_masks).float()  # Attention masks
)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


# Define a directory to save the best model
checkpoint_dir = "checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

/var/folders/rh/1c0lrj_x0x956417g86lc4ph0000gn/T/ipykernel_82559/1218273177.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(attention_masks).float()  # Attention masks


In [8]:
# Initialize Model and Optimizer
model = CSIBERT(feature_dim=feature_dim, num_hidden_layers=6, num_attention_heads = 6)
optimizer = AdamW(model.parameters(), lr=1e-4)

lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=500, num_training_steps=10000
)

loss_fn = nn.MSELoss()

model = model.to(device)

In [ ]:
# Variables to track the best model
best_loss = float('inf')
patience = 10
patience_counter = 0

loss_values = []  # Track loss over iterations
# In[10]:
for epoch in tqdm(range(1000000)):
    model.train()
    epoch_loss = 0
    
    for batch in train_loader:
        inputs, labels, attention_mask = batch  # Add attention mask
        
        inputs = inputs.to(device)
        labels = labels.to(device)
        attention_mask = attention_mask.to(device)
        
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs, attention_mask=attention_mask)
        
        # Compute loss
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        
        lr_scheduler.step()
        
        # Accumulate loss
        epoch_loss += loss.item()
    
    # Average loss for the epoch
    avg_loss = epoch_loss / len(train_loader)
    loss_values.append(avg_loss)
    
    # Check if this is the best model
    if avg_loss < best_loss:
        best_loss = avg_loss
        patience_counter = 0  # Reset patience counter
        # Save the best model
        checkpoint_path = os.path.join(checkpoint_dir, f"best_model.pt")
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            "feature_dim": feature_dim,  # Save feature_dim
            'loss': avg_loss
        }, checkpoint_path)
        print(f"New best model saved with loss {avg_loss:.10f} at epoch {epoch + 1}")
    else:
        patience_counter += 1
    
    # Check if early stopping criteria are met
    if patience_counter >= patience:
        print(f"Early stopping triggered at epoch {epoch + 1}. Best loss: {best_loss:.10f}")
        break

  0%|          | 0/1000000 [00:00<?, ?it/s]

New best model saved with loss 0.3503916948 at epoch 1
New best model saved with loss 0.0281172142 at epoch 2
New best model saved with loss 0.0215394806 at epoch 3
New best model saved with loss 0.0179261808 at epoch 4
New best model saved with loss 0.0160691192 at epoch 5
New best model saved with loss 0.0151808303 at epoch 6
New best model saved with loss 0.0145578961 at epoch 7
New best model saved with loss 0.0139608366 at epoch 8
New best model saved with loss 0.0134003930 at epoch 9
New best model saved with loss 0.0131253135 at epoch 10
New best model saved with loss 0.0126402048 at epoch 11
New best model saved with loss 0.0122836822 at epoch 12
New best model saved with loss 0.0120698611 at epoch 13
New best model saved with loss 0.0120004030 at epoch 14
New best model saved with loss 0.0117868902 at epoch 15
New best model saved with loss 0.0117151182 at epoch 16
New best model saved with loss 0.0116124269 at epoch 17
New best model saved with loss 0.0115408427 at epoch 18
N

In [ ]:
plt.plot(range(1, len(loss_values) + 1), loss_values, marker='o')
plt.xlabel('Epoch')
plt.ylabel('Average Loss')
plt.title('Training Loss Curve')
plt.grid()
plt.show()

In [ ]:
import pandas as pd

# Create a pandas DataFrame
df_loss_vals = pd.DataFrame({
    'Epoch': range(1, len(loss_values) + 1),
    'Loss': loss_values
})

# Save to CSV
df_loss_vals.to_csv('loss_values-12-layers.csv', index=False)

df_loss_vals